# Creación del Pipeline de un Modelo de Machine Learning
En este cuaderno, crearemos un pipeline para el modelo de machine learning. El pipeline incluirá los siguientes pasos:
- Eliminar las columnas que no son necesarias
- Imputar los valores faltantes
- Escalar las características numéricas
- Codificar las características categóricas
- Seleccionar las mejores características
- Entrenar el modelo

Importar librerías

In [1]:
import pickle
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import (
    LinearRegression,
    BayesianRidge,
    SGDRegressor,
    ElasticNet
)


Creación de transformador personalizado para la eliminación de columnas

In [2]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(columns=self.columns)

## Carga de datos

In [3]:
with pd.ExcelFile("data/DatasetTelegramLibro.xlsx") as xlsx:
    sheet = xlsx.sheet_names[0]
    df = pd.read_excel(xlsx, sheet)
df.head()

,Muestra_ID,USERNAME,VIP_credentials,VIP_group,User_status,PASSWORD,MD5,SHA256,SHA512,SHA1,...,Engagement_rate,Mentions,Posts_day,Reposts,Channel_country,FILE_NAME,Results_file,Country_file_name,TOTAL,RISK
0,1.1,pablopiedrabuena@telebot.com,Vip,Ciber,Active,-P#%1NZoH7NZ,b5262d1151b996f12737e49fa2167fbc,a4799b6642c076748357843a07c0d1bcd3e5eb75ccc0d3...,8d10e6af68a519bf194f880940bb803862320564612323...,84fe8d4c552ef52149c7fb0d80060e51275f1847,...,17.58,21,2,10,Other,Ch.txt,1,Other,73.223273,High
1,2.1,davidgarcia@telebot.com,Vip,Ciber,Active,uucp,45b9cfabd8945aa221eb15c629bbf760,730b5353b8edd9b56300b355db37cc39d9da119a68cb1e...,df0f9be2269968c7759fe4e56092f9231f457e5a96a968...,a2dd138f615b6351bd5307c991d6790b5e8c5d9b,...,15.52,25,2,11,Other,France.txt,1,Other,73.827687,High
2,50.1,alvaroafonso@telebot.com,Vip,Ciber,Active,netamin,f350f792fe650c0f858e0cae9cc5a6c7,f860097832f116d4a69b629e712afec9bcba8d3984fec7...,162e3ffef02c08a0b8005b991b2b31db4da76d579f12d4...,0870c592f92eb3d6ba136df5ee4086dd2d0ae21a,...,19.61,21,3,7,Other,Europe.txt,1,Europe,72.340851,High
3,3.1,miguelchacon@telebot.com,Vip,Ciber,Active,pepe,926e27eecdbc7a18858b3798ba99bddd,7c9e7c1494b2684ab7c19d6aff737e460fa9e98d5a234d...,974f3036f39834082e23f4d70f1feba9d4805b3ee2cedb...,265392dc2782778664cc9d56c8e3cd9956661bb0,...,1.00,1,1,1,Other,900K.txt,1,Other,71.606993,High
4,4.1,eduardobejarano@telebot.com,Vip,Ciber,Active,WrMD4fw7,33763d048d98dc38fe9c100067fae7ca,13da9707d7e30adcfa062bf32d8a51219408d86caf5665...,464304b29b44eca6c123853a57b4759268aa2383bb08af...,99cf2501581ed28d097847282e50fab8b5df53b3,...,18.89,68,13,14,Other,44K.txt,1,Other,75.450268,High


In [4]:
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df.columns = df.columns.str.lower()
df['channel'] = df['channel_name']
df['file'] = df['file_name']
#keep
columns = [
    "username",
    "password",
    "channel",
    "file",
    "vip_credentials",
    "vip_group",
    "user_status",
    "md5",
    "sha256",
    "sha512",
    "sha1",
    "password_update",
    "password_type",
    "leaked_password",
    "password_strength",
    "guesses_discover",
    "cracking_time",
    "password_entropy",
    "chat_type",
    "channel_privacity",
    "subscribers",
    "engagement_rate",
    "mentions",
    "posts_day",
    "reposts",
    "channel_country",
    "country_file_name",
    'total'
]
df['vip_credentials'] = df['vip_credentials'].str.lower()
df['vip_group'] = df['vip_group'].str.lower()
df['user_status'] = df['user_status'].str.lower()
df['password_type'] = df['password_type'].str.lower()
df['password_update'] = df['password_update'].str.lower()
df = df[columns]
df

,username,password,channel,file,vip_credentials,vip_group,user_status,md5,sha256,sha512,...,chat_type,channel_privacity,subscribers,engagement_rate,mentions,posts_day,reposts,channel_country,country_file_name,total
0,pablopiedrabuena@telebot.com,-P#%1NZoH7NZ,Cloudata FREE Databases ComboLists,Ch.txt,vip,ciber,active,b5262d1151b996f12737e49fa2167fbc,a4799b6642c076748357843a07c0d1bcd3e5eb75ccc0d3...,8d10e6af68a519bf194f880940bb803862320564612323...,...,Channel,Public,8845,17.58,21,2,10,Other,Other,73.223273
1,davidgarcia@telebot.com,uucp,GODELESS CLOUD,France.txt,vip,ciber,active,45b9cfabd8945aa221eb15c629bbf760,730b5353b8edd9b56300b355db37cc39d9da119a68cb1e...,df0f9be2269968c7759fe4e56092f9231f457e5a96a968...,...,Channel,Public,9393,15.52,25,2,11,Other,Other,73.827687
2,alvaroafonso@telebot.com,netamin,Group,Europe.txt,vip,ciber,active,f350f792fe650c0f858e0cae9cc5a6c7,f860097832f116d4a69b629e712afec9bcba8d3984fec7...,162e3ffef02c08a0b8005b991b2b31db4da76d579f12d4...,...,Channel,Public,2340,19.61,21,3,7,Other,Europe,72.340851
3,miguelchacon@telebot.com,pepe,NoName_Cloud,900K.txt,vip,ciber,active,926e27eecdbc7a18858b3798ba99bddd,7c9e7c1494b2684ab7c19d6aff737e460fa9e98d5a234d...,974f3036f39834082e23f4d70f1feba9d4805b3ee2cedb...,...,Channel,Private,508,1.00,1,1,1,Other,Other,71.606993
4,eduardobejarano@telebot.com,WrMD4fw7,[ REBREND ],44K.txt,vip,ciber,active,33763d048d98dc38fe9c100067fae7ca,13da9707d7e30adcfa062bf32d8a51219408d86caf5665...,464304b29b44eca6c123853a57b4759268aa2383bb08af...,...,Channel,Public,7852,18.89,68,13,14,Other,Other,75.450268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,rafaelortega@telebot.com,d9nW6oGu,DB-MARKEET,UPL.txt,vip,ciber,active,be3f6126764ad36f785f329fea894628,b9896ebab1da8541dbf5cf7c344289b35166fdc0b7a10c...,4f51cea0cb40010c48b2a36a78d56a26eb1639c23c55a3...,...,Channel,Public,5096,17.12,161,4,4,Other,Other,77.449543
193,pablopiedrabuena@telebot.com,Telebot2023+,Databases World🔰,Portugal.txt,vip,ciber,active,4e1afb717d0492f6058984ac34023c16,d6affb3ed382f48e33f496003197dd349d074a11256376...,c657a107790bda8e49f1dd730c8aad929c1a1a6986b4fa...,...,Channel,Public,2244,32.57,3,2,3,Other,Other,74.334728
194,albertoestirado@telebot.com,bee,[ REBREND ],900K.txt,vip,ciber,active,9dfd70fdf15a3cb1ea00d7799ac6651b,62cb81b5904a262ffaeed02abef36bfc540b09f964b8b0...,0159d8667999bcc9e752655a43359c1eaf8958c519faa5...,...,Channel,Public,8129,16.30,74,12,14,Other,Other,79.371522
195,mariadiaz@telebot.com,12345,OPENBULLET,UPL.txt,vip,ciber,active,e2f91f170f15390b5084510e546c09f8,bc47ae33866b036dd2d3c9fa53ce34786087d047c830c8...,be4c43b9eb51e52a4c54fb153f9f13f2f07969adf56ce1...,...,Channel,Public,4974,23.13,260,9,3,Other,Other,78.940622


## Pipeline

Transformación de variables numéricas

In [5]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())
])

Transformación de variables categóricas

In [6]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

Separación de variables independientes de la variable dependiente

In [7]:
X = df.drop('total', axis=1)
y = df['total'].astype('float64').apply(lambda x: x/100)

Descarte de columnas no necesarias

In [8]:
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
unnecesary_features = ['username', 'password', 'md5', 'sha256', 'sha512', 'sha1', 'channel', 'file']
categorical_features = X.select_dtypes(include=['object']).columns
# filtrar características categoricas
categorical_features = [col for col in categorical_features if col not in unnecesary_features]
# filtrar características numericas
numerical_features = [col for col in numerical_features if col not in unnecesary_features]

Creación del transformador de variables

In [9]:
column_transformer = ColumnTransformer(
    transformers=[
        ('numerical', numerical_transformer, numerical_features),
        ('categorical', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

Creación del Pipeline

In [10]:

pipeline = Pipeline(steps=[
    ('drop_columns', DropColumns(columns=unnecesary_features)),
    ('preprocessor', column_transformer),
    ('feature_selection', SelectKBest(f_regression, k=20)),
    ('model', pickle.load(open('model.pkl', 'rb')))
])

Separación de datos de entrenamiento y prueba

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Ajuste del Pipeline con los datos de entrenamiento

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('drop_columns',
                 DropColumns(columns=['username', 'password', 'md5', 'sha256',
                                      'sha512', 'sha1', 'channel', 'file'])),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['leaked_password',
                                                   'password_strength',
                                                   'gue...
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['vip_credentials',
                                                   'vip_group', 'user_status',
                                                   'password_update',
                                                   'password_type', 'chat_type',
                                                   'channel_privacity',
                                                   'channel_country',
                                                   'country_file_name'])])),
                ('feature_selection',
                 SelectKBest(k=20,
                             score_func=<function f_regression at 0x0000025CA547FE20>)),
                ('model', ElasticNet(alpha=0.0001, l1_ratio=0.85))])

## Prueba del Pipeline
Este es el dataset de prueba

In [13]:
pd.concat([X_test, y_test], axis=1).reset_index(drop=True)

,username,password,channel,file,vip_credentials,vip_group,user_status,md5,sha256,sha512,...,chat_type,channel_privacity,subscribers,engagement_rate,mentions,posts_day,reposts,channel_country,country_file_name,total
0,davidgarcia@telebot.com,lennon,𝐹𝑅𝐸𝐸 𝑹𝑫𝑷/𝑽𝑷𝑺 & 𝐵𝐼𝑁𝑁𝐸𝑅𝑆 𝐶𝐶,900K.txt,vip,ciber,active,f4ae294a56d57e0b78e57b5594d272a5,5b75a73245ef49e4e4e4f941c0f9f68a1a3d68b06c10cc...,c5222786d6c1704e440eea79a1861c96389edd2ffc0c2a...,...,Channel,Public,4263,6.27,149,28,1,Other,Other,0.753200
1,yolandagarcia@telebot.com,aiIlyZ_w,Daily Combolist [ Backup ],234K.txt,not vip,otros,active,160259fcbc9376db328c852088598df3,e44519bd9e5923d2d73b661561948111a16fea13a7bd3e...,e236ce050087215f76f1cbdf19dd4cf9226a25eb9f1436...,...,Channel,Public,5527,20.83,21,8,12,Other,Other,0.545916
2,mariadiaz@telebot.com,bin,DB-MARKEET,UPL.txt,vip,ciber,active,c1111bd512b29e821b120b86446026b8,51a1f05af85e342e3c849b47d387086476282d5f50dc24...,b130326ce04125d505761664bf84f3b71836fb198bf39c...,...,Channel,Public,4950,27.08,160,4,2,Other,Other,0.745018
3,yolandasantiago@telebot.com,Telebot2024+,Cloudata FREE Databases ComboLists,India.txt,not vip,otros,active,361f2b3e6e2e7c9ea7bcf59cf5c422b5,436bcf4834b97de8730acbd7c4fe10c977afd797801db9...,97ccc27fb5d01db152871cbbe45db62a997a1caf9e3cc0...,...,Channel,Public,9362,17.59,21,2,10,Other,Other,0.524093
4,yolandacañada@telebot.com,nobody,Fresh Combo mails,Spain.txt,not vip,otros,active,6e854442cd2a940c9e95941dce4ad598,6382b3cc881412b77bfcaeed026001c00d9e3025e66c20...,3a2a5e118c11478d971f896554ac4fc012c5bfbc3f17f8...,...,Channel,Public,2072,10.23,1,3,2,Other,Spain,0.521286
5,miguelchacon@telebot.com,Telebot2022+,Group,Dmk.txt,vip,ciber,active,481833d1a597894e8a0814b4db5ba363,8c23cdae2bafe0678c9a534985f7057980292ea48ae2ab...,512652e09a73f2f081eec7780bf2dfb9377b319cd340e7...,...,Group,Public,6000,0.00,0,0,0,Other,Other,0.718266
6,yolandademiguel@telebot.com,root12345678,Group,Russia.txt,not vip,otros,active,c6285f69b6765a5cdd3b09556d84c5be,1c9c6bef25512438b9ede5f8494269152d0959d8d062b4...,0bf4fa92b796440ea3b704e14aa7758fc8e0a2a37c7df1...,...,Channel,Public,2356,19.82,21,2,8,Other,Other,0.507202
7,pablopiedrabuena@telebot.com,files,Cloudata FREE Databases ComboLists,665K.txt,vip,ciber,active,45b963397aa40d4a0063e0d85e4fe7a1,3d7db37d08f9140fd09f12b9621cd0954b6d56a9d2f357...,975ed4ecf105fba7a632f316949459fe92076756f8612f...,...,Channel,Public,8988,17.69,21,2,10,Other,Other,0.742683
8,yolandaramos@telebot.com,info,HQ COMBO | FRESH BASES | VALID COMBO LIST,France.txt,not vip,otros,active,caf9b6b99962bf5c2264824231d7a40c,06271baf49532c879aa3c58b48671884bcc858f0919741...,653ac8a32dfaa09881d77b31c03a9872e6091edd26a597...,...,Channel,Public,2853,9.50,6,42,2,Other,Other,0.493281
9,angelvillafuertes@telebot.com,money,[ REBREND ],Spain.txt,vip,ciber,active,9726255eec083aa56dc0449a21b33190,8d2ac8b58ead9744d77286de9b0bcb7a894f238c3149fc...,8977fbeabc9c89dba2eca9a66bd2d37df2d21f194b159b...,...,Channel,Public,8129,16.30,74,12,14,Other,Spain,0.806306


Categorías de severidad según el CVSS v3 del NIST

In [14]:
def cvss_score(x):
    if x == 0:
        return 'None'
    elif x < 40:
        return 'Low'
    elif x < 70:
        return 'Medium'
    elif x < 90:
        return 'High'
    else:
        return 'Critical'

Realizar transformaciones y prediccciones con el Pipeline

In [15]:
predictions = pipeline.predict(X_test)
results = pd.concat([X_test[['username', 'password', 'channel', 'file']], y_test], axis=1).reset_index(drop=True)
results['prediction'] = predictions * 100
results['total'] = results['total'] * 100
results['original_severity'] = results['total'].apply(cvss_score)
results['predicted_severity'] = results['prediction'].apply(cvss_score)
results

,username,password,channel,file,total,prediction,original_severity,predicted_severity
0,davidgarcia@telebot.com,lennon,𝐹𝑅𝐸𝐸 𝑹𝑫𝑷/𝑽𝑷𝑺 & 𝐵𝐼𝑁𝑁𝐸𝑅𝑆 𝐶𝐶,900K.txt,75.320002,74.114867,High,High
1,yolandagarcia@telebot.com,aiIlyZ_w,Daily Combolist [ Backup ],234K.txt,54.591556,53.839668,Medium,Medium
2,mariadiaz@telebot.com,bin,DB-MARKEET,UPL.txt,74.501838,75.471171,High,High
3,yolandasantiago@telebot.com,Telebot2024+,Cloudata FREE Databases ComboLists,India.txt,52.409322,53.938651,Medium,Medium
4,yolandacañada@telebot.com,nobody,Fresh Combo mails,Spain.txt,52.128648,53.998562,Medium,Medium
5,miguelchacon@telebot.com,Telebot2022+,Group,Dmk.txt,71.826603,71.956768,High,High
6,yolandademiguel@telebot.com,root12345678,Group,Russia.txt,50.720235,52.875354,Medium,Medium
7,pablopiedrabuena@telebot.com,files,Cloudata FREE Databases ComboLists,665K.txt,74.268331,76.335127,High,High
8,yolandaramos@telebot.com,info,HQ COMBO | FRESH BASES | VALID COMBO LIST,France.txt,49.328056,52.392696,Medium,Medium
9,angelvillafuertes@telebot.com,money,[ REBREND ],Spain.txt,80.630640,79.010778,High,High


Guardar el Pipeline

In [16]:
pickle.dump(pipeline, open('pipeline.pkl', 'wb'))